In [ ]:
import numpy as np
from pyrisk.metric_uncertainty import VolatilityEstimation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from pyrisk.models import lending_club_model
from pyrisk.datasets import lending_club

# loading a dummy model
model = lending_club_model()

# loading original data
data = lending_club(modelling_mode = False)[0]
y = data[['default']]
X = data.drop(['id', 'loan_issue_date','default'], axis = 1)
evaluators =  {'AUC' : [roc_auc_score,'proba'], 'ACC' : [accuracy_score,'class']}
checker = VolatilityEstimation(model, X, y, evaluators)
checker.estimate(0.4,2)

list(checker.metrics_list.keys())

checker.metrics_list['ACC'].shape[0] == 2

np.mean(checker.metrics_list)
np.var(checker.metrics_list)

new_aucs = []

for i in range(0,5000):
    c = np.random.normal(np.mean(checker.auc_list), np.std(checker.auc_list), 1)
    new_aucs.append(c[0])


import matplotlib.pyplot as plt

plt.hist(checker.auc_list, bins=50)
plt.hist(new_aucs, bins=50)

plt.show()

np.var(new_aucs)
np.var(checker.mean_AUCs)


for i in np.arange(0.6, 0.99, 0.01):
    checker.estimate(100, 1-i)

checker.metrics_list